In [1]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
import spacy
from nltk.wsd import lesk
import networkx as nx
nlp = spacy.load("en_core_web_sm")
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# Task 1

class CorpusReader:
    def __init__(self):
        file = open("train.txt", "r")
        f = file.readlines()
        lst = []
        for line in f:
            x = line.strip()
            if x != "":
                lst.append(x)
        inputList = []
        for i in range(0, len(lst), 2):
            newlst = []
            sent = lst[i].split("\t")[1][2 : -2]
            stind_e1 = sent.find("<e1>")
            endind_e1 = sent.find("</e1>")
            e1 = sent[stind_e1 + 4 : endind_e1]
            stind_e2 = sent.find("<e2>")
            endind_e2 = sent.find("</e2>")
            e2 = sent[stind_e2 + 4 : endind_e2]
            newlst.append(sent[:stind_e1] + sent[stind_e1+5:endind_e1] + sent[endind_e1+6:stind_e2] + sent[stind_e2+5:endind_e2] + sent[endind_e2+6:])
            newlst.append(e1.strip())
            newlst.append(e2.strip())
            newlst.append(sent[endind_e1+6:stind_e2-1])
            toplst = []
            top = stind_e1 - 2
            if top >= 0:
                while top >= 0 and sent[top] != " ":
                    top -= 1
                bef = sent[top+1:stind_e1-1]
            else:
                bef = ""
            toplst.append(bef)
            top = endind_e2 + 6
            if top < len(sent):
                while top < len(sent) and sent[top] != " ":
                    top += 1
                aftr = sent[endind_e2+6:top]
            else:
                aftr = ""
            toplst.append(aftr)
            newlst.append(toplst)
            if lst[i + 1] == "no_relation":
                newlst.append(None)
                newlst.append("no_relation")
            else:
                i1 = lst[i + 1].find(":")
                i2 = lst[i + 1].find("(")
                if(lst[i + 1][i2 + 2] == '1'):
                    newlst.append(True)
                else:
                    newlst.append(False)
                newlst.append(lst[i + 1][i1 + 1 : i2])
            inputList.append(newlst)
        self.inputList = inputList
        file = open("test.txt", "r")
        f = file.readlines()
        lst = []
        for line in f:
            x = line.strip()
            if x != "":
                lst.append(x)
        inputList = []
        for i in range(0, len(lst), 2):
            newlst = []
            sent = lst[i].split("\t")[1][2 : -2]
            stind_e1 = sent.find("<e1>")
            endind_e1 = sent.find("</e1>")
            e1 = sent[stind_e1 + 4 : endind_e1]
            stind_e2 = sent.find("<e2>")
            endind_e2 = sent.find("</e2>")
            e2 = sent[stind_e2 + 4 : endind_e2]
            newlst.append(sent[:stind_e1] + sent[stind_e1+5:endind_e1] + sent[endind_e1+6:stind_e2] + sent[stind_e2+5:endind_e2] + sent[endind_e2+6:])
            newlst.append(e1.strip())
            newlst.append(e2.strip())
            newlst.append(sent[endind_e1+6:stind_e2-1])
            toplst = []
            top = stind_e1 - 2
            if top >= 0:
                while top >= 0 and sent[top] != " ":
                    top -= 1
                bef = sent[top+1:stind_e1-1]
            else:
                bef = ""
            toplst.append(bef)
            top = endind_e2 + 6
            if top < len(sent):
                while top < len(sent) and sent[top] != " ":
                    top += 1
                aftr = sent[endind_e2+6:top]
            else:
                aftr = ""
            toplst.append(aftr)
            newlst.append(toplst)
            if lst[i + 1] == "no_relation":
                newlst.append(None)
                newlst.append("no_relation")
            else:
                i1 = lst[i + 1].find(":")
                i2 = lst[i + 1].find("(")
                if(lst[i + 1][i2 + 2] == '1'):
                    newlst.append(True)
                else:
                    newlst.append(False)
                newlst.append(lst[i + 1][i1 + 1 : i2])
            inputList.append(newlst)
        self.inputList2 = inputList

# inputList is a list of [Sentence, e1, e2, Sentence b/w entities, before and after word of e1 n e2, Direction of relation, Name of relation] lists

cr = CorpusReader()


In [3]:
# converts test sentence into the form we want to process our model with

def createlist_ts(sent):
    testList = []
    newlst = []
    sent = sent[1 : -1].strip()
    stind_e1 = sent.find("<e1>")
    endind_e1 = sent.find("</e1>")
    e1 = sent[stind_e1 + 4 : endind_e1]
    stind_e2 = sent.find("<e2>")
    endind_e2 = sent.find("</e2>")
    e2 = sent[stind_e2 + 4 : endind_e2]
    newlst.append(sent[:stind_e1] + sent[stind_e1+5:endind_e1] + sent[endind_e1+6:stind_e2] + sent[stind_e2+5:endind_e2] + sent[endind_e2+6:])
    newlst.append(e1.strip())
    newlst.append(e2.strip())
    newlst.append(sent[endind_e1+6:stind_e2-1])
    toplst = []
    top = stind_e1 - 2
    if top >= 0:
        while top >= 0 and sent[top] != " ":
            top -= 1
        bef = sent[top+1:stind_e1-1]
    else:
        bef = ""
    toplst.append(bef)
    top = endind_e2 + 6
    if top < len(sent):
        while top < len(sent) and sent[top] != " ":
            top += 1
        aftr = sent[endind_e2+6:top]
    else:
        aftr = ""
    toplst.append(aftr)
    newlst.append(toplst)
    testList.append(newlst)
    return testList

In [4]:
# Task 2

# e1Tokens is a list of e1 Tokens lists and e2Tokens is a list of e2 Tokens lists

def entities_tokens(list1):
    e1Tokens = []
    e2Tokens = []
    for ind in list1:
        e1Tokens.append(word_tokenize(ind[1]))
        e2Tokens.append(word_tokenize(ind[2]))
    return e1Tokens, e2Tokens

In [5]:
e1Tokens_train, e2Tokens_train = entities_tokens(cr.inputList)
e1Tokens_test, e2Tokens_test = entities_tokens(cr.inputList2)

In [6]:
# lemmas is a list of sentence's lemmas lists

def lemmas_sentences(list1):
    lemmas = []
    for ind in list1:
        sent = ind[0]
        lm = nltk.word_tokenize(sent)
        nl = []
        for r in lm:
            lemmatizer = WordNetLemmatizer()
            nl.append(lemmatizer.lemmatize(r))
        lemmas.append(nl)
    return lemmas

In [7]:
# pos is a list of POS Tags lists

def pos_sentences(list1):
    pos = []
    for ind in list1:
        sent = ind[0]
        lm = nltk.pos_tag(nltk.word_tokenize(sent))
        nl = []
        for r in lm:
            tag = r[1]
            nl.append(tag)
        pos.append(nl)
    return pos

In [8]:
# deps is a list of dependency b/w 2 tokens and the root of it lists

def deps_sentences(list1):
    deps = []
    for ind in list1:
        sent = ind[0] 
        doc = nlp(sent)
        lm = []
        for tkn in doc:
            lm.append([tkn.dep_, tkn.head.text])
        deps.append(lm)
    return deps

In [9]:
deps_train = deps_sentences(cr.inputList)
deps_test = deps_sentences(cr.inputList2)

In [10]:
# hypernyms, hyponyms, meronyms and holonyms are list of lists

def hhmh_sentences(list1):
    hypernyms = []
    hyponyms = []
    meronyms = []
    holonyms = [] 
    for ind in list1:
        sent = ind[0]
        tkns = word_tokenize(sent)
        tmp1 = []
        tmp2 = []
        tmp3 = []
        tmp4 = []
        for tkn in tkns:
            syn = lesk(tkns, tkn)
            if syn is not None:
                for hypernym in syn.hypernyms():
                    tmp1.append(hypernym)
                for hyponym in syn.hyponyms():
                    tmp2.append(hyponym)
                for meronym in syn.part_meronyms():
                    tmp3.append(meronym)
                for holonym in syn.part_holonyms():
                    tmp4.append(holonym)
        hypernyms.append(tmp1)
        hyponyms.append(tmp2)
        meronyms.append(tmp3)
        holonyms.append(tmp4)
    return hypernyms, hyponyms, meronyms, holonyms

In [11]:
# e1pos n e2pos are lists of pos tag lists of e1 n e2

def e1pos_e2pos(list1, list2):
    e1pos = []
    e2pos = []

    for tkns in list1:
        lm = nltk.pos_tag(tkns)
        nl = []
        for r in lm:
            tag = r[1]
            nl.append(tag)
        e1pos.append(nl)

    for tkns in list2:
        lm = nltk.pos_tag(tkns)
        nl = []
        for r in lm:
            tag = r[1]
            nl.append(tag)
        e2pos.append(nl)
    return e1pos, e2pos

In [12]:
e1pos_train, e2pos_train = e1pos_e2pos(e1Tokens_train, e2Tokens_train)
e1pos_test, e2pos_test = e1pos_e2pos(e1Tokens_test, e2Tokens_test)

In [13]:
#bwwords_pos is a list of pos tags of words b/w entities lists

def bwwords_pos_entities(list1):
    bwwords_pos = [] 
    for ind in list1:
        sentbwwrds = ind[3]
        lm = nltk.pos_tag(nltk.word_tokenize(sentbwwrds))
        nl = []
        for r in lm:
            tag = r[1]
            nl.append(tag)
        bwwords_pos.append(nl)
    return bwwords_pos

In [14]:
bwwords_pos_train = bwwords_pos_entities(cr.inputList)
bwwords_pos_test = bwwords_pos_entities(cr.inputList2)

In [15]:
# noOfWords is a list of no of words b/w entities lists

def noOfWords_entities(list1):
    noOfWords = []
    for ind in list1:
        sent = ind[3]
        noOfWords.append(len(sent.split(" ")))
    return noOfWords

In [16]:
noOfWords_train = noOfWords_entities(cr.inputList)
noOfWords_test = noOfWords_entities(cr.inputList2)

In [17]:
# Shortest Dependency Path

def sdp_sentences(list1):
    sdp = []

    for ind in list1:
        sent = ind[0]
        doc = nlp(sent)
        edges = []
        for tkn in doc:
            for chld in tkn.children:
                edges.append(('{0}'.format(tkn.lower_), '{0}'.format(chld.lower_)))
        graph = nx.Graph(edges)
        entity1 = ""
        entity2 = ""
        ent1wds = ind[1].lower().split(" ")
        ent2wds = ind[2].lower().split(" ")
        for wds in ent1wds:
            if wds in graph.nodes:
                entity1 = wds
                break
        for wds in ent2wds:
            if wds in graph.nodes:
                entity2 = wds
                break
        if entity1 == "" or entity2 == "":
            sdp.append([])
        else:
            try:
                sdp.append(nx.shortest_path(graph, source = entity1, target = entity2))
            except:
                sdp.append([])
    return sdp

In [18]:
sdp_train = sdp_sentences(cr.inputList)
sdp_test = sdp_sentences(cr.inputList2)

In [19]:
# e1hypernyms and e2hypernyms are list of e1 n e2 hypernyms lists

def e1e2_hypernyms(list1):
    e1hypernyms = []
    e2hypernyms = []

    for ind in list1:
        e1 = ind[1]
        tkns = word_tokenize(e1)
        tmp1 = []
        for tkn in tkns:
            syn = lesk(tkns, tkn)
            if syn is not None:
                hyp = syn.hypernyms()
                if hyp:
                    tmp1.append(hyp[0])
        if not tmp1:
            e1hypernyms.append(tmp1)
        else: 
            e1hypernyms.append(tmp1[-1:])
        e2 = ind[2]
        tkns = word_tokenize(e2)
        tmp1 = []
        for tkn in tkns:
            syn = lesk(tkns, tkn)
            if syn is not None:
                hyp = syn.hypernyms()
                if hyp:
                    tmp1.append(hyp[0])
        if not tmp1:
            e2hypernyms.append(tmp1)
        else: 
            e2hypernyms.append(tmp1[-1:])
    return e1hypernyms, e2hypernyms

In [20]:
e1hypernyms_train, e2hypernyms_train = e1e2_hypernyms(cr.inputList)
e1hypernyms_test, e2hypernyms_test = e1e2_hypernyms(cr.inputList2)

In [21]:
# e1ner and e2ner is a list of ner tags of e1 n e2

def e1e2_ner(list1):
    e1ner = []
    e2ner = []

    for ind in list1:
        sent = ind[1]
        doc = nlp(sent)
        lm = []
        for tkn in doc.ents:
            lm.append([tkn.label_])
        e1ner.append(lm)
        sent = ind[2]
        doc = nlp(sent)
        lm = []
        for tkn in doc.ents:
            lm.append([tkn.label_])
        e2ner.append(lm)
    return e1ner, e2ner

In [22]:
e1ner_train, e2ner_train = e1e2_ner(cr.inputList)
e1ner_test, e2ner_test = e1e2_ner(cr.inputList2)

In [23]:
# Task 3

def unique(list1): 
    unique_list = []  
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

def list_to_string_original(list1):
    newlst = []
    for x in list1:
        newlst.append("-".join(x))
    return newlst

def list_to_string(list1):
    newlst = []
    for x in list1:
        newlst.append("-".join(x))
    newlst.append("*****")
    return newlst

def assign_indices(list1):
    dic1 = {}
    cnt = 0
    for x in list1:
        dic1[x] = cnt
        cnt += 1
    dic2 = {}
    cnt = 0
    for x in list1:
        dic2[cnt] = x
        cnt += 1
    return dic1, dic2

def listoflist_to_string_original(list1):
    newlst = []
    for x in list1:
        tmp = []
        for i in x:
            if len(i) > 0:
                tmp.append(i[0])
        newlst.append("-".join(tmp))
    return newlst

def listoflist_to_string(list1):
    newlst = []
    for x in list1:
        tmp = []
        for i in x:
            if len(i) > 0:
                tmp.append(i[0])
        newlst.append("-".join(tmp))
    newlst.append("*****")
    return newlst

def hypernyms_to_string_original(list1):
    newlst = []
    for x in list1:
        if not x:
            strg = ""
        else:
            strg = x[0].name()
        newlst.append(strg)
    return newlst

def hypernyms_to_string(list1):
    newlst = []
    for x in list1:
        if not x:
            strg = ""
        else:
            strg = x[0].name()
        newlst.append(strg)
    newlst.append("*****")
    return newlst

def convert_to_string(dirtion, rel):
    tmp = []
    for x in range(len(dirtion)):
        tmp.append(rel[x] + f" {dirtion[x]}")
    return tmp

def get_items(list1, tt):
    e1 = []
    for ind in list1:
        e1.append(ind[tt])
    return e1

def get_integer_encoded(lst, dic):
    integer_encoded = []
    for x in lst:
        if x not in dic.keys():
            strp = dic["*****"]
        else:
            strp = dic[x]
        integer_encoded.append(strp)
    return integer_encoded

def get_onehot_encoded(lst1, lst2):
    onehot_encoded = list()
    for value in lst1:
        letter = [0 for _ in range(len(lst2))]
        letter[value] = 1
        onehot_encoded.append(letter)
    return onehot_encoded

def flatten_test_sentence():
    xw_test = []
    for ind in range(len(testList)):
        tmp = []
        for x in onehot_encoded_e1words_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_e2words_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_bww_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_e1_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_e2_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_bwwords_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_befaft_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_noOfWords_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_e1hyp_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_e2hyp_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_e1ner_test_sentence[ind]:
            tmp.append(x)
        for x in onehot_encoded_e1ner_test_sentence[ind]:
            tmp.append(x)
        xw_test.append(tmp)
    return xw_test

def construct_csr(list1):
    row = []
    col = []
    data = []

    for i in range(len(list1)):
        for j in range(len(list1[0])):
            if list1[i][j] == 1:
                row.append(i)
                col.append(j)
                data.append(1)

    X_train = csr_matrix((data, (row, col)), shape=(len(list1), len(list1[0])))
    return X_train

def predict_y(feat_name):
    if feat_name == "*****":
        print("Sorry, our model is unable to predict the relation between the entities in the test sentence!")
    else:
        lst = feat_name.split(" ")
        rel = lst[0]
        dirc = lst[1]
        if rel == "no_relation":
            print("There is no relation between the entities in the test sentence!")
        else:
            if dirc == 'True':
                print(f"The relation between entities in the test sentence is {rel}(e1, e2)")
            elif dirc == 'False':
                print(f"The relation between entities in the test sentence is {rel}(e2, e1)")

In [24]:
# 1-hot encoding of e1 n e2 words
# train

e1_string = get_items(cr.inputList, 1)
e2_string = get_items(cr.inputList, 2)    
e1unique_string = unique(e1_string)
e2unique_string = unique(e2_string)
e1unique_string.append("*****")
e2unique_string.append("*****")
e1wordslist_to_int, e1wordsint_to_list = assign_indices(e1unique_string)
e2wordslist_to_int, e2wordsint_to_list = assign_indices(e2unique_string)
integer_encoded_e1words = [e1wordslist_to_int[s] for s in e1_string]
integer_encoded_e2words = [e2wordslist_to_int[s] for s in e2_string]
onehot_encoded_e1words = get_onehot_encoded(integer_encoded_e1words, e1unique_string)
onehot_encoded_e2words = get_onehot_encoded(integer_encoded_e2words, e2unique_string)

In [25]:
# 1-hot encoding of e1 n e2 words
# test

def onehotencoding_e1e2Words(list1):
    e1_string_test = get_items(list1, 1)
    e2_string_test = get_items(list1, 2)
    integer_encoded_e1words_test = get_integer_encoded(e1_string_test, e1wordslist_to_int)
    integer_encoded_e2words_test = get_integer_encoded(e2_string_test, e2wordslist_to_int)
    onehot_encoded_e1words_test = get_onehot_encoded(integer_encoded_e1words_test, e1unique_string)
    onehot_encoded_e2words_test = get_onehot_encoded(integer_encoded_e2words_test, e2unique_string)
    return e1_string_test, e2_string_test, integer_encoded_e1words_test, integer_encoded_e2words_test, onehot_encoded_e1words_test, onehot_encoded_e2words_test

In [26]:
e1_string_test, e2_string_test, integer_encoded_e1words_test, integer_encoded_e2words_test, onehot_encoded_e1words_test, onehot_encoded_e2words_test = onehotencoding_e1e2Words(cr.inputList2)

In [27]:
# 1-hot encoding of words b/w e1 n e2
# train

bw_words_string = get_items(cr.inputList, 3)   
bwwords_unique_string = unique(bw_words_string)
bwwords_unique_string.append("*****")
bwwlist_to_int, bwwint_to_list = assign_indices(bwwords_unique_string)
integer_encoded_bww = [bwwlist_to_int[s] for s in bw_words_string]
onehot_encoded_bww = get_onehot_encoded(integer_encoded_bww, bwwords_unique_string)

In [28]:
# 1-hot encoding of words b/w e1 n e2
# test

def onehotencoding_betweenWords(list1):
    bw_words_string_test = get_items(list1, 3)
    integer_encoded_bww_test = get_integer_encoded(bw_words_string_test, bwwlist_to_int)
    onehot_encoded_bww_test = get_onehot_encoded(integer_encoded_bww_test, bwwords_unique_string)
    return bw_words_string_test, integer_encoded_bww_test, onehot_encoded_bww_test 

In [29]:
bw_words_string_test, integer_encoded_bww_test, onehot_encoded_bww_test = onehotencoding_betweenWords(cr.inputList2)

In [30]:
# 1-hot encoding of e1pos n e2pos
# train

e1uniquepos = unique(e1pos_train)
e2uniquepos = unique(e2pos_train)
e1pos_string = list_to_string_original(e1pos_train)
e2pos_string = list_to_string_original(e2pos_train)
e1uniquepos_string = list_to_string(e1uniquepos)
e2uniquepos_string = list_to_string(e2uniquepos)
e1list_to_int, e1int_to_list = assign_indices(e1uniquepos_string)
e2list_to_int, e2int_to_list = assign_indices(e2uniquepos_string)
integer_encoded_e1 = [e1list_to_int[s] for s in e1pos_string]
integer_encoded_e2 = [e2list_to_int[s] for s in e2pos_string]
onehot_encoded_e1 = get_onehot_encoded(integer_encoded_e1, e1uniquepos_string)
onehot_encoded_e2 = get_onehot_encoded(integer_encoded_e2, e2uniquepos_string)

In [31]:
# 1-hot encoding of e1pos n e2pos
# test

def onehotencoding_e1e2pos(var1, var2):
    e1pos_string_test = list_to_string_original(var1)
    e2pos_string_test = list_to_string_original(var2)
    integer_encoded_e1_test = get_integer_encoded(e1pos_string_test, e1list_to_int)
    integer_encoded_e2_test = get_integer_encoded(e2pos_string_test, e2list_to_int)
    onehot_encoded_e1_test = get_onehot_encoded(integer_encoded_e1_test, e1uniquepos_string)
    onehot_encoded_e2_test = get_onehot_encoded(integer_encoded_e2_test, e2uniquepos_string)
    return e1pos_string_test, e2pos_string_test, integer_encoded_e1_test, integer_encoded_e2_test, onehot_encoded_e1_test, onehot_encoded_e2_test 

In [32]:
e1pos_string_test, e2pos_string_test, integer_encoded_e1_test, integer_encoded_e2_test, onehot_encoded_e1_test, onehot_encoded_e2_test = onehotencoding_e1e2pos(e1pos_test, e2pos_test) 

In [33]:
# 1-hot encoding of bwwords_pos
# train

bwwords_unique_pos = unique(bwwords_pos_train)
bwwords_pos_string = list_to_string_original(bwwords_pos_train)
bwwords_unique_pos_string = list_to_string(bwwords_unique_pos)
bwwordslist_to_int, bwwordsint_to_list = assign_indices(bwwords_unique_pos_string)
integer_encoded_bwwords = [bwwordslist_to_int[s] for s in bwwords_pos_string]
onehot_encoded_bwwords = get_onehot_encoded(integer_encoded_bwwords, bwwords_unique_pos_string)

In [34]:
# 1-hot encoding of bwwords_pos
# test

def onehotencoding_betweenwordspos(var1):
    bwwords_pos_string_test = list_to_string_original(var1)
    integer_encoded_bwwords_test = get_integer_encoded(bwwords_pos_string_test, bwwordslist_to_int)
    onehot_encoded_bwwords_test = get_onehot_encoded(integer_encoded_bwwords_test, bwwords_unique_pos_string)
    return bwwords_pos_string_test, integer_encoded_bwwords_test, onehot_encoded_bwwords_test

In [35]:
bwwords_pos_string_test, integer_encoded_bwwords_test, onehot_encoded_bwwords_test = onehotencoding_betweenwordspos(bwwords_pos_test)

In [36]:
# 1-hot encoding of before and after words of entities
# train

bef_aft_words = get_items(cr.inputList, 4)
unique_bef_aft_words = unique(bef_aft_words)
bef_aft_words_string = list_to_string_original(bef_aft_words)
unique_bef_aft_words_string = list_to_string(unique_bef_aft_words)
befaftlist_to_int, befaftint_to_list = assign_indices(unique_bef_aft_words_string)
integer_encoded_befaft = [befaftlist_to_int[s] for s in bef_aft_words_string]
onehot_encoded_befaft = get_onehot_encoded(integer_encoded_befaft, unique_bef_aft_words_string)

In [37]:
# 1-hot encoding of before and after words of entities
# test

def onehotencoding_befaftwords(list1):
    bef_aft_words_test = get_items(list1, 4)
    bef_aft_words_string_test = list_to_string_original(bef_aft_words_test)
    integer_encoded_befaft_test = get_integer_encoded(bef_aft_words_string_test, befaftlist_to_int)
    onehot_encoded_befaft_test = get_onehot_encoded(integer_encoded_befaft_test, unique_bef_aft_words_string)
    return bef_aft_words_test, bef_aft_words_string_test, integer_encoded_befaft_test, onehot_encoded_befaft_test

In [38]:
bef_aft_words_test, bef_aft_words_string_test, integer_encoded_befaft_test, onehot_encoded_befaft_test = onehotencoding_befaftwords(cr.inputList2)

In [39]:
# 1-hot encoding of noOfWords
# train

unique_noOfWords = unique(noOfWords_train)
unique_noOfWords.append("*****")
noOfWords_list_to_int = dict((c, i) for i, c in enumerate(unique_noOfWords))
noOfWords_int_to_list = dict((i, c) for i, c in enumerate(unique_noOfWords))
integer_encoded_noOfWords = [noOfWords_list_to_int[c] for c in noOfWords_train]
onehot_encoded_noOfWords = get_onehot_encoded(integer_encoded_noOfWords, unique_noOfWords)

In [40]:
# 1-hot encoding of noOfWords
# test

def onehotencoding_noOfWords(var1):
    integer_encoded_noOfWords_test = get_integer_encoded(var1, noOfWords_list_to_int)
    onehot_encoded_noOfWords_test = get_onehot_encoded(integer_encoded_noOfWords_test, unique_noOfWords)
    return integer_encoded_noOfWords_test, onehot_encoded_noOfWords_test

In [41]:
integer_encoded_noOfWords_test, onehot_encoded_noOfWords_test = onehotencoding_noOfWords(noOfWords_test)

In [42]:
# 1-hot encoding of e1hypernyms and e2hypernyms
# train

e1unique_hyp = unique(e1hypernyms_train)
e2unique_hyp = unique(e2hypernyms_train)
e1hyp_string = hypernyms_to_string_original(e1hypernyms_train)
e2hyp_string = hypernyms_to_string_original(e2hypernyms_train)
e1unique_hyp_string = hypernyms_to_string(e1unique_hyp)
e2unique_hyp_string = hypernyms_to_string(e2unique_hyp)
e1hyplist_to_int, e1hypint_to_list = assign_indices(e1unique_hyp_string)
e2hyplist_to_int, e2hypint_to_list = assign_indices(e2unique_hyp_string)
integer_encoded_e1hyp = [e1hyplist_to_int[s] for s in e1hyp_string]
integer_encoded_e2hyp = [e2hyplist_to_int[s] for s in e2hyp_string]
onehot_encoded_e1hyp = get_onehot_encoded(integer_encoded_e1hyp, e1unique_hyp_string)
onehot_encoded_e2hyp = get_onehot_encoded(integer_encoded_e2hyp, e2unique_hyp_string)

In [43]:
# 1-hot encoding of e1hypernyms and e2hypernyms
# test

def onehotencoding_e1e2hypernyms(var1, var2):
    e1hyp_string_test = hypernyms_to_string_original(var1)
    e2hyp_string_test = hypernyms_to_string_original(var2)
    integer_encoded_e1hyp_test = get_integer_encoded(e1hyp_string_test, e1hyplist_to_int)
    integer_encoded_e2hyp_test = get_integer_encoded(e2hyp_string_test, e2hyplist_to_int)
    onehot_encoded_e1hyp_test = get_onehot_encoded(integer_encoded_e1hyp_test, e1unique_hyp_string)
    onehot_encoded_e2hyp_test = get_onehot_encoded(integer_encoded_e2hyp_test, e2unique_hyp_string)
    return e1hyp_string_test, e2hyp_string_test, integer_encoded_e1hyp_test, integer_encoded_e2hyp_test, onehot_encoded_e1hyp_test, onehot_encoded_e2hyp_test

In [44]:
e1hyp_string_test, e2hyp_string_test, integer_encoded_e1hyp_test, integer_encoded_e2hyp_test, onehot_encoded_e1hyp_test, onehot_encoded_e2hyp_test = onehotencoding_e1e2hypernyms(e1hypernyms_test, e2hypernyms_test)

In [45]:
# 1-hot encoding of e1ner n e2ner
# train

e1unique_ner = unique(e1ner_train)
e2unique_ner = unique(e2ner_train)
e1ner_string = listoflist_to_string_original(e1ner_train)
e2ner_string = listoflist_to_string_original(e2ner_train)
e1unique_ner_string = listoflist_to_string(e1unique_ner)
e2unique_ner_string = listoflist_to_string(e2unique_ner)
e1nerlist_to_int, e1nerint_to_list = assign_indices(e1unique_ner_string)
e2nerlist_to_int, e2nerint_to_list = assign_indices(e2unique_ner_string)
integer_encoded_e1ner = [e1nerlist_to_int[s] for s in e1ner_string]
integer_encoded_e2ner = [e2nerlist_to_int[s] for s in e2ner_string]
onehot_encoded_e1ner = get_onehot_encoded(integer_encoded_e1ner, e1unique_ner_string)    
onehot_encoded_e2ner = get_onehot_encoded(integer_encoded_e2ner, e2unique_ner_string)

In [46]:
# 1-hot encoding of e1ner n e2ner
# test

def onehotencoding_e1e2ner(var1, var2):
    e1ner_string_test = listoflist_to_string_original(var1)
    e2ner_string_test = listoflist_to_string_original(var2)
    integer_encoded_e1ner_test = get_integer_encoded(e1ner_string_test, e1nerlist_to_int)
    integer_encoded_e2ner_test = get_integer_encoded(e2ner_string_test, e2nerlist_to_int)
    onehot_encoded_e1ner_test = get_onehot_encoded(integer_encoded_e1ner_test, e1unique_ner_string)
    onehot_encoded_e2ner_test = get_onehot_encoded(integer_encoded_e2ner_test, e2unique_ner_string)
    return e1ner_string_test, e2ner_string_test, integer_encoded_e1ner_test, integer_encoded_e2ner_test, onehot_encoded_e1ner_test, onehot_encoded_e2ner_test

In [47]:
e1ner_string_test, e2ner_string_test, integer_encoded_e1ner_test, integer_encoded_e2ner_test, onehot_encoded_e1ner_test, onehot_encoded_e2ner_test = onehotencoding_e1e2ner(e1ner_test, e2ner_test)

In [48]:
# 1-hot encoding of direction of relation and relation name without direction and with direction
# train

def onehotencoding_ytrain(list1):
    direction = get_items(list1, 5)
    relation_name = get_items(list1, 6)
    unique_relations = unique(relation_name)
    unique_relations.append("*****")
    ysetlist_to_int_wo, ysetint_to_list_wo = assign_indices(unique_relations)
    onehot_encoded_yset_wo = [ysetlist_to_int_wo[s] for s in relation_name]
    yset = convert_to_string(direction, relation_name)
    yset_unique = unique(yset)
    yset_unique.append("*****")
    ysetlist_to_int_w, ysetint_to_list_w = assign_indices(yset_unique)
    onehot_encoded_yset_w = [ysetlist_to_int_w[s] for s in yset]
    return direction, relation_name, unique_relations, ysetlist_to_int_wo, ysetint_to_list_wo, onehot_encoded_yset_wo, yset, yset_unique, ysetlist_to_int_w, ysetint_to_list_w, onehot_encoded_yset_w 

In [49]:
direction, relation_name, unique_relations, ysetlist_to_int_wo, ysetint_to_list_wo, onehot_encoded_yset_wo, yset, yset_unique, ysetlist_to_int_w, ysetint_to_list_w, onehot_encoded_yset_w = onehotencoding_ytrain(cr.inputList) 

In [50]:
# 1-hot encoding of direction of relation and relation name without direction and with direction
# test

def onehotencoding_ytest(list1):
    direction_test = get_items(list1, 5)
    relation_name_test = get_items(list1, 6)
    onehot_encoded_yset_test_wo = get_integer_encoded(relation_name_test, ysetlist_to_int_wo)
    yset_test = convert_to_string(direction_test, relation_name_test)
    onehot_encoded_yset_test_w = get_integer_encoded(yset_test, ysetlist_to_int_w)
    return direction_test, relation_name_test, onehot_encoded_yset_test_wo, yset_test, onehot_encoded_yset_test_w  

In [51]:
direction_test, relation_name_test, onehot_encoded_yset_test_wo, yset_test, onehot_encoded_yset_test_w = onehotencoding_ytest(cr.inputList2)

In [52]:
# xw_train, xw_test

xw_train = []

for ind in range(len(cr.inputList)):
    tmp = []
    for x in onehot_encoded_e1words[ind]:
        tmp.append(x)
    for x in onehot_encoded_e2words[ind]:
        tmp.append(x)
    for x in onehot_encoded_bww[ind]:
        tmp.append(x)
    for x in onehot_encoded_e1[ind]:
        tmp.append(x)
    for x in onehot_encoded_e2[ind]:
        tmp.append(x)
    for x in onehot_encoded_bwwords[ind]:
        tmp.append(x)
    for x in onehot_encoded_befaft[ind]:
        tmp.append(x)
    for x in onehot_encoded_noOfWords[ind]:
        tmp.append(x)
    for x in onehot_encoded_e1hyp[ind]:
        tmp.append(x)
    for x in onehot_encoded_e2hyp[ind]:
        tmp.append(x)
    for x in onehot_encoded_e1ner[ind]:
        tmp.append(x)
    for x in onehot_encoded_e1ner[ind]:
        tmp.append(x)
    xw_train.append(tmp)

xw_test = []

for ind in range(len(cr.inputList2)):
    tmp = []
    for x in onehot_encoded_e1words_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_e2words_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_bww_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_e1_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_e2_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_bwwords_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_befaft_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_noOfWords_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_e1hyp_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_e2hyp_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_e1ner_test[ind]:
        tmp.append(x)
    for x in onehot_encoded_e1ner_test[ind]:
        tmp.append(x)
    xw_test.append(tmp)

In [53]:
# Task 4

# X_train and X_test

X_train = construct_csr(xw_train)
X_test = construct_csr(xw_test)

In [54]:
# Y_train and Y_test without considering direction

Y_train = onehot_encoded_yset_wo
Y_test = onehot_encoded_yset_test_wo

In [55]:
# SVM classifier without considering direction

mdl_wo = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, Y_train)
mdl_wo.fit(X_train, Y_train)
accuracy_mdl_wo = mdl_wo.score(X_test, Y_test)
print(f"\nAccuracy of our model without considering direction: {accuracy_mdl_wo * 100} %")
y_pred_wo = mdl_wo.predict(X_test)
print("\nMacro Precision, Recall and F-scores and Support of our model without considering direction:\n")
print(precision_recall_fscore_support(Y_test, y_pred_wo))


Accuracy of our model without considering direction: 45.40381791483113 %

Macro Precision, Recall and F-scores and Support of our model without considering direction:

(array([0.33333333, 0.56267409, 0.51754386, 0.47058824, 0.605     ,
       0.3566879 , 0.45578231, 0.60747664, 0.2114094 , 0.42447917,
       0.60215054, 0.89380531, 0.52083333, 0.71794872, 0.52      ,
       0.60465116, 0.52941176, 0.74193548]), array([0.60387324, 0.66229508, 0.36875   , 0.1       , 0.53070175,
       0.29015544, 0.536     , 0.37572254, 0.150358  , 0.61278195,
       0.44444444, 0.73722628, 0.18382353, 0.78504673, 0.30409357,
       0.45614035, 0.1011236 , 0.70769231]), array([0.42955542, 0.60843373, 0.43065693, 0.16494845, 0.56542056,
       0.32      , 0.49264706, 0.46428571, 0.17573222, 0.50153846,
       0.51141553, 0.808     , 0.27173913, 0.75      , 0.38376384,
       0.52      , 0.16981132, 0.72440945]), array([568, 305, 160,  80, 228, 193, 125, 173, 419, 266, 126, 137, 136,
       107, 171,  57

In [56]:
# Y_train and Y_test considering direction

Y_train = onehot_encoded_yset_w
Y_test = onehot_encoded_yset_test_w

In [57]:
# SVM classifier considering direction

mdl_w = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, Y_train)
mdl_w.fit(X_train, Y_train)
accuracy_mdl_w = mdl_w.score(X_test, Y_test)
print(f"\nAccuracy of our model considering direction: {accuracy_mdl_w * 100} %")
y_pred_w = mdl_w.predict(X_test)
print("\nMacro Precision, Recall and F-scores and Support of our model considering direction:\n")
print(precision_recall_fscore_support(Y_test, y_pred_w))


Accuracy of our model considering direction: 44.22907488986784 %

Macro Precision, Recall and F-scores and Support of our model considering direction:

(array([0.34615385, 0.54285714, 0.58333333, 0.33333333, 0.36266667,
       0.53932584, 0.38095238, 0.28767123, 0.42553191, 0.57142857,
       0.21212121, 0.33082707, 0.5       , 0.91891892, 0.5       ,
       0.89361702, 0.7721519 , 0.48076923, 0.67123288, 0.46666667,
       0.45851528, 0.44680851, 0.59090909, 0.60683761, 0.42857143,
       0.22222222, 0.52941176, 0.64285714, 0.        , 0.625     ,
       0.35820896, 0.45762712, 0.63636364, 0.75      , 0.84      ]), array([0.5591716 , 0.74347826, 0.4       , 0.16666667, 0.59130435,
       0.5106383 , 0.14545455, 0.23333333, 0.49382716, 0.36363636,
       0.23389021, 0.45360825, 0.39393939, 0.86075949, 0.16901408,
       0.72413793, 0.89705882, 0.27472527, 0.52688172, 0.23333333,
       0.62130178, 0.47727273, 0.2       , 0.52985075, 0.15      ,
       0.05263158, 0.33333333, 0.2857142

In [58]:
test_sentence = input("Please enter a valid test sentence - \" De <e1> La Salle High School </e1> was founded by the <e2> Christian Brothers </e2> . \"")
testList = createlist_ts(test_sentence)
print("\nWe've stored the test sentence in the following form:")
print(f"\n{testList}\n")

Please enter a valid test sentence - " De <e1> La Salle High School </e1> was founded by the <e2> Christian Brothers </e2> . "" <e1> Tunisia </e1> 's flag carrier <e2> Tunisair </e2> scored 60 million Tunisian dinars ( 41.15 million US dollars ) in net income in 2009 despite world economic downturn which badly hit air transport industry the company said on Friday . "

We've stored the test sentence in the following form:

[["Tunisia 's flag carrier Tunisair scored 60 million Tunisian dinars ( 41.15 million US dollars ) in net income in 2009 despite world economic downturn which badly hit air transport industry the company said on Friday .", 'Tunisia', 'Tunisair', "'s flag carrier", ['', 'scored']]]



In [59]:
e1Tokens_test_sentence, e2Tokens_test_sentence = entities_tokens(testList)
print("\nThe tokens of entity e1 are:")
print(f"\n{e1Tokens_test_sentence}\n")
print("\nThe tokens of entity e2 are:")
print(f"\n{e2Tokens_test_sentence}\n")


The tokens of entity e1 are:

[['Tunisia']]


The tokens of entity e2 are:

[['Tunisair']]



In [60]:
lemmas_test_sentence = lemmas_sentences(testList)
print("\nThe lemmas in the sentence are:")
print(f"\n{lemmas_test_sentence}\n")


The lemmas in the sentence are:

[['Tunisia', "'s", 'flag', 'carrier', 'Tunisair', 'scored', '60', 'million', 'Tunisian', 'dinar', '(', '41.15', 'million', 'US', 'dollar', ')', 'in', 'net', 'income', 'in', '2009', 'despite', 'world', 'economic', 'downturn', 'which', 'badly', 'hit', 'air', 'transport', 'industry', 'the', 'company', 'said', 'on', 'Friday', '.']]



In [61]:
pos_test_sentence = pos_sentences(testList)
print("\nThe pos tags in the sentence are:")
print(f"\n{pos_test_sentence}\n")


The pos tags in the sentence are:

[['NNP', 'POS', 'NN', 'NN', 'NNP', 'VBD', 'CD', 'CD', 'JJ', 'NNS', '(', 'CD', 'CD', 'NNP', 'NNS', ')', 'IN', 'JJ', 'NN', 'IN', 'CD', 'IN', 'NN', 'JJ', 'NN', 'WDT', 'RB', 'VBD', 'NN', 'NN', 'NN', 'DT', 'NN', 'VBD', 'IN', 'NNP', '.']]



In [62]:
deps_test_sentence = deps_sentences(testList)
print("\nThe dependencies in the sentence are:")
print(f"\n{deps_test_sentence}\n")


The dependencies in the sentence are:

[[['poss', 'carrier'], ['case', 'Tunisia'], ['compound', 'carrier'], ['nsubj', 'scored'], ['nsubj', 'scored'], ['ROOT', 'scored'], ['compound', 'million'], ['nummod', 'dinars'], ['amod', 'dinars'], ['dobj', 'scored'], ['punct', 'dinars'], ['compound', 'million'], ['nummod', 'dollars'], ['compound', 'dollars'], ['appos', 'dinars'], ['punct', 'dinars'], ['prep', 'scored'], ['amod', 'income'], ['pobj', 'in'], ['prep', 'scored'], ['pobj', 'in'], ['prep', 'scored'], ['nmod', 'downturn'], ['amod', 'downturn'], ['pobj', 'despite'], ['nsubj', 'hit'], ['advmod', 'hit'], ['relcl', 'downturn'], ['compound', 'transport'], ['compound', 'industry'], ['dobj', 'hit'], ['det', 'company'], ['nsubj', 'said'], ['relcl', 'downturn'], ['prep', 'said'], ['pobj', 'on'], ['punct', 'scored']]]



In [63]:
hypernyms_test_sentence, hyponyms_test_sentence, meronyms_test_sentence, holonyms_test_sentence = hhmh_sentences(testList)
print("\nThe hypernyms in the sentence are:")
print(f"\n{hypernyms_test_sentence}\n")
print("\nThe hyponyms in the sentence are:")
print(f"\n{hyponyms_test_sentence}\n")
print("\nThe meronyms in the sentence are:")
print(f"\n{meronyms_test_sentence}\n")
print("\nThe holonyms in the sentence are:")
print(f"\n{holonyms_test_sentence}\n")


The hypernyms in the sentence are:

[[Synset('list.n.01'), Synset('organism.n.01'), Synset('notch.v.01'), Synset('large_integer.n.01'), Synset('large_integer.n.01'), Synset('african.n.01'), Synset('tunisian_monetary_unit.n.01'), Synset('large_integer.n.01'), Synset('monetary_unit.n.01'), Synset('metallic_element.n.01'), Synset('income.n.01'), Synset('financial_gain.n.01'), Synset('metallic_element.n.01'), Synset('disregard.n.02'), Synset('worsening.n.02'), Synset('succeed.v.01'), Synset('region.n.01'), Synset('move.v.02'), Synset('commercial_enterprise.n.01'), Synset('complement.n.03'), Synset('express.v.02'), Synset('weekday.n.01')]]


The hyponyms in the sentence are:

[[Synset('line.v.04'), Synset('scarify.v.02'), Synset('scotch.v.02'), Synset('scribe.v.01'), Synset('australian_dollar.n.01'), Synset('bahamian_dollar.n.01'), Synset('barbados_dollar.n.01'), Synset('belize_dollar.n.01'), Synset('bermuda_dollar.n.01'), Synset('brunei_dollar.n.01'), Synset('canadian_dollar.n.01'), Synse

In [64]:
e1ner_test_sentence, e2ner_test_sentence = e1e2_ner(testList)
print("\nThe NER tags of entity e1 are:")
print(f"\n{e1ner_test_sentence}\n")
print("\nThe NER tags of entity e2 are:")
print(f"\n{e2ner_test_sentence}\n")


The NER tags of entity e1 are:

[[]]


The NER tags of entity e2 are:

[[]]



In [65]:
e1pos_test_sentence, e2pos_test_sentence = e1pos_e2pos(e1Tokens_test_sentence, e2Tokens_test_sentence)
print("\nThe POS tags of tokens of entity e1 are:")
print(f"\n{e1pos_test_sentence}\n")
print("\nThe POS tags of tokens of entity e2 are:")
print(f"\n{e2pos_test_sentence}\n")


The POS tags of tokens of entity e1 are:

[['NN']]


The POS tags of tokens of entity e2 are:

[['NN']]



In [66]:
bwwords_pos_test_sentence = bwwords_pos_entities(testList)
print("\nThe POS tags of tokens between entities e1 and e2 are:")
print(f"\n{bwwords_pos_test_sentence}\n")


The POS tags of tokens between entities e1 and e2 are:

[['POS', 'NN', 'NN']]



In [67]:
noOfWords_test_sentence = noOfWords_entities(testList)
print("\nThe number of words between entities e1 and e2 is:")
print(f"\n{noOfWords_test_sentence}\n")


The number of words between entities e1 and e2 is:

[3]



In [68]:
e1hypernyms_test_sentence, e2hypernyms_test_sentence = e1e2_hypernyms(testList)
print("\nThe hypernyms of entity e1 in the sentence are:")
print(f"\n{e1hypernyms_test_sentence}\n")
print("\nThe hypernyms of entity e2 in the sentence are:")
print(f"\n{e2hypernyms_test_sentence}\n")


The hypernyms of entity e1 in the sentence are:

[[]]


The hypernyms of entity e2 in the sentence are:

[[]]



In [69]:
e1_string_test_sentence, e2_string_test_sentence, integer_encoded_e1words_test_sentence, integer_encoded_e2words_test_sentence, onehot_encoded_e1words_test_sentence, onehot_encoded_e2words_test_sentence = onehotencoding_e1e2Words(testList)
bw_words_string_test_sentence, integer_encoded_bww_test_sentence, onehot_encoded_bww_test_sentence = onehotencoding_betweenWords(testList)
e1pos_string_test_sentence, e2pos_string_test_sentence, integer_encoded_e1_test_sentence, integer_encoded_e2_test_sentence, onehot_encoded_e1_test_sentence, onehot_encoded_e2_test_sentence = onehotencoding_e1e2pos(e1pos_test_sentence, e2pos_test_sentence) 
bwwords_pos_string_test_sentence, integer_encoded_bwwords_test_sentence, onehot_encoded_bwwords_test_sentence = onehotencoding_betweenwordspos(bwwords_pos_test_sentence)
bef_aft_words_test_sentence, bef_aft_words_string_test_sentence, integer_encoded_befaft_test_sentence, onehot_encoded_befaft_test_sentence = onehotencoding_befaftwords(testList)
integer_encoded_noOfWords_test_sentence, onehot_encoded_noOfWords_test_sentence = onehotencoding_noOfWords(noOfWords_test_sentence)
e1hyp_string_test_sentence, e2hyp_string_test_sentence, integer_encoded_e1hyp_test_sentence, integer_encoded_e2hyp_test_sentence, onehot_encoded_e1hyp_test_sentence, onehot_encoded_e2hyp_test_sentence = onehotencoding_e1e2hypernyms(e1hypernyms_test_sentence, e2hypernyms_test_sentence)
e1ner_string_test_sentence, e2ner_string_test_sentence, integer_encoded_e1ner_test_sentence, integer_encoded_e2ner_test_sentence, onehot_encoded_e1ner_test_sentence, onehot_encoded_e2ner_test_sentence = onehotencoding_e1e2ner(e1ner_test_sentence, e2ner_test_sentence)


In [70]:
# X_test_sentence

xw_ts = flatten_test_sentence()
X_test_sentence = construct_csr(xw_ts)

In [71]:
# Prediction

Y_test_sent_pred = mdl_w.predict(X_test_sentence)
feat_name = ysetint_to_list_w.get(Y_test_sent_pred[0])
predict_y(feat_name)

The relation between entities in the test sentence is country_of_headquarters(e2, e1)
